In [29]:
%pip install pokerkit==2024.3.0

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pokerkit==2024.3.0 (from versions: 0.0.0, 0.0.1, 0.0.2, 0.1.0, 0.1.1, 0.2.0, 0.2.1, 0.3.0, 0.3.1, 0.3.2, 0.4.0, 0.4.1, 0.4.2, 0.4.3, 0.4.4, 0.4.5, 0.4.6, 0.4.7, 0.4.8, 0.4.9, 0.4.10, 0.4.11, 0.4.12, 0.4.13, 0.4.14, 0.4.15, 0.4.16, 0.4.17, 0.5.0a0, 0.5.0a1, 0.5.0, 0.5.1, 0.5.2, 0.5.3, 0.5.4, 0.6.0a0, 0.6.0, 0.6.1, 0.6.2, 0.6.3)
ERROR: No matching distribution found for pokerkit==2024.3.0


In [16]:
from pokerkit import *

In [17]:
from random import sample
from itertools import product, combinations
import copy


In [18]:
from concurrent.futures import ProcessPoolExecutor

def oddsCalc(cards, board): #'AsKs', 'Kc8h8d'
    with ProcessPoolExecutor() as executor:
        return calculate_hand_strength(2, parse_range(cards),Card.parse(board), 2,  5, Deck.STANDARD, (StandardHighHand,), sample_count=2000, executor=executor,)

In [19]:
oddsCalc('AsAd', 'AhAc2c2d6h')

1.0

In [20]:
def Nstreet(board):
    #returns number of street we at
    girth = len(board)/2
    return 3 - (girth>=3) - (girth>=4)
    

In [21]:
from math import *
def move(cards, board, stack, pot, aggressivity_index=1, odds_manual=None):
    #Allows custom odds to test
    if odds_manual != None:
        odds = odds_manual
    else:
        odds = oddsCalc(cards, board)
    print(odds)
    
    #ensures no division by 0 (if odds 50/50 (1-2*odds)=0)
    if (1-2*odds):
        tl = -pot + 2*stack*odds - stack
        tr = (pot**2 + 2*pot*stack + stack**2*(1-2*odds)**2)**0.5
        kelly_solutions = (tl +tr)/2, (tl-tr)/2
    else:
        kelly_solutions = -1, -1
    
    #finds max call+bet using kelly solutions
    callMax = (-(pot+stack-2*odds*stack)+((pot+stack-2*odds*stack)**2+4*odds*pot*stack)**0.5)/2  #, (-(pot+stack-2*odds*stack)-((pot+stack-2*odds*stack)**2+4*odds*pot*stack)**0.5)/2
    if kelly_solutions[0] + kelly_solutions[1] > 0:
        print(kelly_solutions)
        bet = round(kelly_solutions[0])
        # EV version: bet, callMax = round(kelly_solutions[0]), inf
    else:
        bet = 0
        # EV version: bet, callMax = 0, floor((odds*pot)/(1-2*odds))
    print(kelly_solutions)
    return aggressivity_index*bet, aggressivity_index*callMax/Nstreet(board) #is this the correct move? should i divide by the street? should i leave bet intact?


In [17]:
board = '9hTs9s'

Nstreet(board)


    

3

In [20]:
print(move('7c2s', '', 100, 50, .499999))
#something is broken

0.35525
(18.270351544518277, -97.22035154451828)
(0.0, 3.0450525006358657)


In [ ]:
o = 0.001
p = 100
s = 500

b0, b1 = (-(p+s-2*o*s)+((p+s-2*o*s)**2+4*o*p*s)**0.5)/2, (-(p+s-2*o*s)-((p+s-2*o*s)**2+4*o*p*s)**0.5)/2
b0, b1

(0.08346082519307174, -599.083460825193)

In [5]:
import tkinter as tk
from tkinter import simpledialog, messagebox
from pokerkit import NoLimitTexasHoldem, Automation

# Your bot function
def move(hole, board, stack, pot):
    return 50 if len(board) < 5 else 0  # Dummy logic

class PokerGame:
    def __init__(self, root):
        self.root = root
        self.root.title("PokerKit GUI Poker")

        self.automations = (
            Automation.ANTE_POSTING,
            Automation.BET_COLLECTION,
            Automation.BLIND_OR_STRADDLE_POSTING,
            Automation.CARD_BURNING,
            Automation.HOLE_CARDS_SHOWING_OR_MUCKING,
            Automation.HAND_KILLING,
            Automation.CHIPS_PUSHING,
            Automation.CHIPS_PULLING,
        )

        self.setup_gui()
        self.root.after(100, self.new_game)  # Delay game start

    def setup_gui(self):
        self.status = tk.Label(self.root, text="", font=("Arial", 14))
        self.status.pack()

        self.hand_label = tk.Label(self.root, text="Your Hand: ", font=("Arial", 16))
        self.hand_label.pack()

        self.board_label = tk.Label(self.root, text="Board: ", font=("Arial", 16))
        self.board_label.pack()

        self.stack_label = tk.Label(self.root, text="", font=("Arial", 14))
        self.stack_label.pack()

        frame = tk.Frame(self.root)
        frame.pack(pady=10)

        self.btn_check = tk.Button(frame, text="Check", command=self.check)
        self.btn_bet = tk.Button(frame, text="Bet", command=self.bet)
        self.btn_call = tk.Button(frame, text="Call", command=self.call)
        self.btn_fold = tk.Button(frame, text="Fold", command=self.fold)

        self.btn_check.pack(side=tk.LEFT, padx=5)
        self.btn_bet.pack(side=tk.LEFT, padx=5)
        self.btn_call.pack(side=tk.LEFT, padx=5)
        self.btn_fold.pack(side=tk.LEFT, padx=5)

    def new_game(self):
        self.state = NoLimitTexasHoldem.create_state(
            self.automations,
            False,
            0,
            (5, 10),
            10,
            (1000, 1000),
            2,
        )

        # Deal 2 hole cards to each player
        for _ in range(2):
            for _ in range(self.state.player_count):
                self.state.deal_hole()

        self.advance_until_action()

    def advance_all_the_way(self):
        while self.state.status == "chance":
            self.state.proceed()

    def advance_until_action(self):
        self.advance_all_the_way()
        self.update_gui()
        self.next_action()

    def update_gui(self):
        board = ''.join(str(card) for card in self.state.board_cards)
        hole = ''.join(str(card) for card in self.state.hole_cards[0]) if self.state.hole_cards and self.state.hole_cards[0] else 'Waiting...'
        pot = sum(self.state.bets)
        stacks = self.state.stacks

        self.hand_label.config(text=f"Your Hand: {hole}")
        self.board_label.config(text=f"Board: {board}")
        self.stack_label.config(text=f"Pot: {pot} | You: {stacks[0]} | Bot: {stacks[1]}")
        self.status.config(text=f"{'Your' if self.state.actor_index == 0 else 'Bot'} turn")

    def disable_all(self):
        for b in [self.btn_check, self.btn_bet, self.btn_call, self.btn_fold]:
            b.config(state=tk.DISABLED)

    def enable_buttons(self):
        self.btn_check.config(state=tk.NORMAL if self.state.can_check_or_call() else tk.DISABLED)
        self.btn_bet.config(state=tk.NORMAL if self.state.can_complete_bet_or_raise_to() else tk.DISABLED)
        self.btn_call.config(state=tk.NORMAL if self.state.can_check_or_call() else tk.DISABLED)
        self.btn_fold.config(state=tk.NORMAL if self.state.can_fold() else tk.DISABLED)

    def next_action(self):
        if self.state.status == 'over':
            hand_0 = self.state.hands[0]
            hand_1 = self.state.hands[1]
            board = self.state.board_cards

            val_0 = hand_0.evaluate_hand(board)
            val_1 = hand_1.evaluate_hand(board)

            if val_0 > val_1:
                result = "You win!"
            elif val_1 > val_0:
                result = "Bot wins!"
            else:
                result = "It's a tie!"

            messagebox.showinfo("Hand Over", result)
            self.root.after(1000, self.new_game)
            return

        if self.state.actor_index == 1:
            self.disable_all()
            self.root.after(500, self.bot_action)
        else:
            self.enable_buttons()

    def check(self):
        self.state.check_or_call()
        self.advance_until_action()

    def bet(self):
        max_bet = self.state.stacks[0]
        amount = simpledialog.askinteger("Bet", "How much?", minvalue=1, maxvalue=max_bet)
        if amount:
            self.state.complete_bet_or_raise_to(amount)
            self.advance_until_action()

    def call(self):
        self.state.check_or_call()
        self.advance_until_action()

    def fold(self):
        self.state.fold()
        self.advance_until_action()

    def bot_action(self):
        hole = ''.join(str(c) for c in self.state.hole_cards[1])
        board = ''.join(str(c) for c in self.state.board_cards)
        stack = self.state.stacks[1]
        pot = sum(self.state.bets)
        amount = move(hole, board, stack, pot)

        try:
            if amount == 0 and self.state.can_check_or_call():
                self.state.check_or_call()
            elif amount > 0 and self.state.can_complete_bet_or_raise_to():
                self.state.complete_bet_or_raise_to(amount)
            elif self.state.can_check_or_call():
                self.state.check_or_call()
            elif self.state.can_fold():
                self.state.fold()
        except Exception as e:
            print(f"Bot error: {e}")
            self.state.fold()

        self.advance_until_action()

# Run GUI
root = tk.Tk()
app = PokerGame(root)
root.mainloop()


In [ ]:
#add agressivity index
#include check percentage
#adjust to a per street basis cuz this is same as assuming opponent goes all in everytime